# タイタニック号の乗客生存予測

In [8]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
import time
import os

---

# ワークスペースへの接続とデータセットの取得

In [9]:
# データセットの取得とデータフレームへの変換
df=pd.read_csv('titanic.csv')
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


## 前処理

In [10]:
# データの型の確認
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [11]:
# 欠損値の確認
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [12]:
# 不要な特徴の削除
# PassengerId: id であるため削除
# Name: 性別や階級を示唆する敬称の情報は有益であると考えれるが、手間がかかるので今回は削除
# Ticket: 規則性を見出しにくいので削除
# Cabin: 欠損値の数が多すぎるので削除
drop_list = ['PassengerId', 'Name', 'Ticket', 'Cabin']
df = df.drop(drop_list, axis = 1)

In [13]:
# 簡単に埋められる欠損値の処理

# 'S'の度数が最も多いため'S'で埋める
df.Embarked=df.Embarked.fillna('S') 

# 無難に中央値で埋める
df.Age=df.Age.fillna(df['Age'].median())

In [14]:
# Pclassを文字列に変換（数値型ではget_dummiesが上手く実行できないため）
df['Pclass'] = df['Pclass'].astype('str')

In [15]:
# 質的変数のワンホットベクトルを得る
df = pd.get_dummies(df)

In [16]:
# AgeとFareの標準化を行う
from sklearn import preprocessing
ss = preprocessing.StandardScaler()
df[['Age', 'Fare']] = ss.fit_transform(df[['Age', 'Fare']])

In [17]:
df.head(3)

,Survived,Age,SibSp,Parch,Fare,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,0,-0.565736,1,0,-0.502445,0,0,1,0,1,0,0,1
1,1,0.663861,1,0,0.786845,1,0,0,1,0,1,0,0
2,1,-0.258337,0,0,-0.488854,0,0,1,1,0,0,0,1


In [18]:
# 計算時間をかけるために、データ量を増やす
df = pd.concat([df,df],axis=0)
df = pd.concat([df,df],axis=0)
df = pd.concat([df,df],axis=0)
df = pd.concat([df,df],axis=0)
df = pd.concat([df,df],axis=0)

In [19]:
# 目的変数と説明変数を分離
x_df=df.drop('Survived',axis = 1)
y_df=df['Survived']

In [20]:
x_df.head(3)

,Age,SibSp,Parch,Fare,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,-0.565736,1,0,-0.502445,0,0,1,0,1,0,0,1
1,0.663861,1,0,0.786845,1,0,0,1,0,1,0,0
2,-0.258337,0,0,-0.488854,0,0,1,1,0,0,0,1


In [21]:
# train/test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x_df, y_df, test_size=0.2, random_state=12)

## CPUのコア数を確認

In [22]:
os.cpu_count()

8

---

## 学習

In [23]:
def run(n_jobs):
    print("n_jobs=", n_jobs)
    start_time = time.time()
    model = RandomForestClassifier(n_estimators=1000, n_jobs=n_jobs) # 計算時間をかけるために、あえて大きなn_estimatorsにしておく
    model.fit(X_train, y_train)
    end_time = time.time()
    print ("elapsed_time:{:.2f}".format(end_time - start_time) + "[sec]")
    
run(n_jobs=1) 
run(n_jobs=2)
run(n_jobs=4)

n_jobs= 1
elapsed_time:12.03[sec]
n_jobs= 2
elapsed_time:6.14[sec]
n_jobs= 4
elapsed_time:4.21[sec]
